In [8]:
from datetime import datetime
import time
start = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
start_dt = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
	
time.sleep(5)
end = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
end_dt = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
print(end_dt-start_dt)

0:00:05


# WADI数据集处理

In [4]:
import pandas as pd
import os
import copy
import numpy as np
def convertNumpy(df):
    # 每隔10取一行数据，从第四列开始取值
    x = df[df.columns[2:]].values[::10, :]
    # print(f"x.shape: {x.shape}")
    return (x - x.min(0)) / (x.ptp(0) + 1e-4)  # 归一化，x.ptp(0) = x.max(0) - x.min(0)，添加1e-4防止除0

def convertNumpy_4(df):
    batch_num = 4
    # Select every 10th row starting from the 4th row
    df = df.iloc[batch_num::10, :]
    # Select columns starting from the 4th column
    x = df.iloc[:, :]
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    batch_data = (x - min_vals) / (max_vals - min_vals + 1e-4)
    # print(f"batch_data.shape: {batch_data.shape}")
    return batch_data  # Normalized data

def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a
dataset_folder = 'data/WADI/2019'
# 训练数据,跳过前1000行,读取20万行
train_df = pd.read_csv(os.path.join(dataset_folder, 'WADI_14days_new.csv'),index_col=0,nrows=2e5)
test_df = pd.read_csv(os.path.join(dataset_folder, 'WADI_attackdataLABLE.csv'),header=1,index_col=0).iloc[:172800,2:]
train = train_df.copy(deep=True)
test = test_df.copy(deep=True)
label = test['Attack LABLE (1:No Attack, -1:Attack)'].replace({1: 0, -1: 1}).copy(deep = True)
test = test_df.drop('Attack LABLE (1:No Attack, -1:Attack)', axis=1).copy(deep = True)
# 将label 拓展成与test相同的维度
label = pd.concat([label]*len(test.columns), axis=1)
train.dropna(how='all', inplace=True); test.dropna(how='all', inplace=True)	# 删除全为NaN的行
train.fillna(0, inplace=True); test.fillna(0, inplace=True)	# 将NaN替换为0



train = convertNumpy(train)
test = convertNumpy_4(test)
labels = convertNumpy_4(label)


_, min_a, max_a = normalize3(np.concatenate((train, test), axis=0))
train, _, _ = normalize3(train, min_a, max_a)
test, _, _ = normalize3(test, min_a, max_a)

output_folder = 'processed/WADI1'
os.makedirs(output_folder, exist_ok=True)
print(f"train.shape: {train.shape}, test.shape: {test.shape}, label.shape: {label.shape}")
for file in ['train', 'test', 'labels']:
	np.save(os.path.join(output_folder, f'{file}.npy'), eval(file))

train.shape: (20000, 127), test.shape: (17280, 127), label.shape: (172800, 127)


# SWaT数据集

In [24]:
import pandas as pd
import os
import numpy as np

def convertNumpy(df):
    # 每隔10取一行数据，从第二列开始取值
    x = df[df.columns[1:]].values[::10, :]
    # print(f"x.shape: {x.shape}")
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    return (x - min_vals) / (max_vals - min_vals + 1e-4)  # 归一化，x.ptp(0) = x.max(0) - x.min(0)，添加1e-4防止除0
def convertNumpy_4(df):
    batch_num = 20
    # Select every 10th row starting from the 4th row
    df = df.iloc[batch_num::60, :]
    # Select columns starting from the 4th column
    x = df.iloc[:, :]
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    batch_data = (x - min_vals) / (max_vals - min_vals + 1e-4)
    # print(f"batch_data.shape: {batch_data.shape}")
    return batch_data  # Normalized data
def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a
# 读取数据
dataset_folder = 'data/SWaT'
train_df = pd.read_csv(os.path.join(dataset_folder, 'SWaT_Dataset_Attack_v0.csv'), nrows=int(2e5))
test_df = pd.read_csv(os.path.join(dataset_folder, 'SWaT_Dataset_Attack_v0.csv'))
# 处理训练数据和测试数据
train = train_df.copy(deep=True).iloc[:, :-1]
test = test_df.copy(deep=True).iloc[:, 1:]




# 标签处理
label = test['Normal/Attack'].apply(lambda x: 1 if x in ['Attack', 'A ttack'] else 0)
test = test.drop('Normal/Attack', axis=1).copy(deep=True)

# 将标签扩展为与 test 相同的维度
label = pd.concat([label] * len(test.columns), axis=1)

# 删除全为NaN的行，并填充NaN为0
train.dropna(how='all', inplace=True); test.dropna(how='all', inplace=True)	# 删除全为NaN的行
train.fillna(0, inplace=True); test.fillna(0, inplace=True)	# 将NaN替换为0
# 确保数据是数值类型
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')



# 执行归一化
train, test, labels = convertNumpy(train), convertNumpy_4(test), convertNumpy_4(label)
_, min_a, max_a = normalize3(np.concatenate((train, test), axis=0))
train, _, _ = normalize3(train, min_a, max_a)
test, _, _ = normalize3(test, min_a, max_a)

output_folder = 'processed/SWaT1'
print(f"train.shape: {train.shape}, test.shape: {test.shape}, labels.shape: {labels.shape}")
# 保存数据为.npy文件
for file in ['train', 'test', 'labels']:
    np.save(os.path.join(output_folder, f'{file}.npy'), eval(file))


train.shape: (20000, 51), test.shape: (7499, 51), labels.shape: (7499, 51)


# PSM数据集

In [22]:
import pandas as pd
import os
import numpy as np

def convertNumpy(df):
    # 每隔10取一行数据，从第二列开始取值
    x = df[df.columns[:]].values[::5, :]
    # print(f"x.shape: {x.shape}")
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    return (x - min_vals) / (max_vals - min_vals + 1e-4)  # 归一化，x.ptp(0) = x.max(0) - x.min(0)，添加1e-4防止除0
def convertNumpy_4(df):
    x = df.iloc[int(len(df)*0.8):, :]
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    batch_data = (x - min_vals) / (max_vals - min_vals + 1e-4)
    # print(f"batch_data.shape: {batch_data.shape}")
    return batch_data  # Normalized data
def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a
# 读取数据
dataset_folder = 'data/PSM'
train_df = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
test_df = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))
label_df = pd.read_csv(os.path.join(dataset_folder, 'test_label.csv'))

train = train_df.copy(deep=True).iloc[:132480, 1:]
test = test_df.copy(deep=True).iloc[:87840, 1:]
label = label_df.copy(deep=True).iloc[:87840, 1]


train.dropna(how='all', inplace=True); test.dropna(how='all', inplace=True)	# 删除全为NaN的行
train.fillna(0, inplace=True); test.fillna(0, inplace=True)	# 将NaN替换为0

_, min_a, max_a = normalize3(np.concatenate((train, test), axis=0))
train, _, _ = normalize3(train, min_a, max_a)
test, _, _ = normalize3(test, min_a, max_a)


# 将标签扩展为与 test 相同的维度
label = pd.concat([label] * len(test.columns), axis=1)

# train = convertNumpy(train)
test, labels = convertNumpy_4(test), convertNumpy_4(label)
# labels = label

# output_folder = 'data/PSM/npy'
output_folder = 'processed/PSM'
os.makedirs(output_folder, exist_ok=True)
print(f"train.shape: {train.shape}, test.shape: {test.shape}, labels.shape: {labels.shape}")
# 保存数据为.npy文件
for file in ['train', 'test', 'labels']:
    np.save(os.path.join(output_folder, f'{file}.npy'), eval(file))

train.shape: (132480, 25), test.shape: (17568, 25), labels.shape: (17568, 25)


# HVAC数据集

In [5]:
import pandas as pd
import os
import numpy as np

def convertNumpy(df):
    # 每隔10取一行数据，从第二列开始取值
    x = df[df.columns[:]].values[::5, :]
    # print(f"x.shape: {x.shape}")
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    return (x - min_vals) / (max_vals - min_vals + 1e-4)  # 归一化，x.ptp(0) = x.max(0) - x.min(0)，添加1e-4防止除0
def convertNumpy_4(df):
    x = df.iloc[int(len(df)*0.8):, :]
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    batch_data = (x - min_vals) / (max_vals - min_vals + 1e-4)
    # print(f"batch_data.shape: {batch_data.shape}")
    return batch_data  # Normalized data
def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a

# 读取数据
dataset_folder = 'data/HVAC'
train_df = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
test_df = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))
label_df = pd.read_csv(os.path.join(dataset_folder, 'test_label.csv'))
# print(f"train_df.shape: {train_df.shape}, test_df.shape: {test_df.shape}, label_df.shape: {label_df.shape}")
# 处理训练数据和测试数据
train = train_df.copy(deep=True).iloc[:, 1:]
test = test_df.copy(deep=True).iloc[:, 1:]
label = label_df.copy(deep=True)
_, min_a, max_a = normalize3(np.concatenate((train, test), axis=0))
train, _, _ = normalize3(train, min_a, max_a)
test, _, _ = normalize3(test, min_a, max_a)

# 确保数据是数值类型
# train = train.apply(pd.to_numeric, errors='coerce')
# test = test.apply(pd.to_numeric, errors='coerce')
# label = label.apply(pd.to_numeric, errors='coerce')
# 将标签扩展为与 test 相同的维度
label = pd.concat([label] * len(test.columns), axis=1)
# train = convertNumpy(train)
# test, labels = convertNumpy_4(test), convertNumpy_4(label)

labels = label
print(f"train.shape: {train.shape}, test.shape: {test.shape}, labels.shape: {labels.shape}")
output_folder = 'data/HVAC/npy'
os.makedirs(output_folder, exist_ok=True)
# 保存数据为.npy文件
for file in ['train', 'test', 'labels']:
    np.save(os.path.join(output_folder, f'{file}.npy'), eval(file))

train.shape: (6476, 21), test.shape: (1636, 21), labels.shape: (1636, 21)


# ELS数据集

In [15]:
import pandas as pd
import os
import numpy as np

def convertNumpy(df):
    # 每隔10取一行数据，从第二列开始取值
    x = df[df.columns[:]].values[::5, :]
    # print(f"x.shape: {x.shape}")
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    return (x - min_vals) / (max_vals - min_vals + 1e-4)  # 归一化，x.ptp(0) = x.max(0) - x.min(0)，添加1e-4防止除0
def convertNumpy_4(df):
    x = df.iloc[int(len(df)*0.8):, :]
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    batch_data = (x - min_vals) / (max_vals - min_vals + 1e-4)
    # print(f"batch_data.shape: {batch_data.shape}")
    return batch_data  # Normalized data
def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a

# 读取数据
dataset_folder = 'data/els'
train_df = pd.read_csv(os.path.join(dataset_folder, 'train1.csv'))
test_df = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))
label_df = pd.read_csv(os.path.join(dataset_folder, 'test_label.csv'))
# print(f"train_df.shape: {train_df.shape}, test_df.shape: {test_df.shape}, label_df.shape: {label_df.shape}")
# 处理训练数据和测试数据
train = train_df.copy(deep=True).iloc[:, 1:]
test = test_df.copy(deep=True).iloc[:, 1:]
label = label_df.copy(deep=True)
_, min_a, max_a = normalize3(np.concatenate((train, test), axis=0))
train, _, _ = normalize3(train, min_a, max_a)
test, _, _ = normalize3(test, min_a, max_a)

# 确保数据是数值类型
# train = train.apply(pd.to_numeric, errors='coerce')
# test = test.apply(pd.to_numeric, errors='coerce')
# label = label.apply(pd.to_numeric, errors='coerce')
# 将标签扩展为与 test 相同的维度
label = pd.concat([label] * len(test.columns), axis=1)
# train = convertNumpy(train)
# test, labels = convertNumpy_4(test), convertNumpy_4(label)
labels = label
print(f"train.shape: {train.shape}, test.shape: {test.shape}, labels.shape: {labels.shape}")
output_folder = 'data/els/npy'
output_folder = 'processed/els'
os.makedirs(output_folder, exist_ok=True)
# 保存数据为.npy文件
for file in ['train', 'test', 'labels']:
    np.save(os.path.join(output_folder, f'{file}.npy'), eval(file))

train.shape: (6865, 17), test.shape: (1786, 17), labels.shape: (1786, 17)


# hydrogen数据集

In [3]:
import pandas as pd
import os
import numpy as np

def convertNumpy(df):
    # 每隔10取一行数据，从第二列开始取值
    x = df[df.columns[:]].values[::5, :]
    # print(f"x.shape: {x.shape}")
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    return (x - min_vals) / (max_vals - min_vals + 1e-4)  # 归一化，x.ptp(0) = x.max(0) - x.min(0)，添加1e-4防止除0
def convertNumpy_4(df):
    x = df.iloc[int(len(df)*0.8):, :]
    min_vals = x.min(axis=0)
    max_vals = x.max(axis=0)
    batch_data = (x - min_vals) / (max_vals - min_vals + 1e-4)
    # print(f"batch_data.shape: {batch_data.shape}")
    return batch_data  # Normalized data
def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a

# 读取数据
dataset_folder = 'data/hydrogen'
train_df = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
test_df = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))
label_df = pd.read_csv(os.path.join(dataset_folder, 'test_label.csv'))
# print(f"train_df.shape: {train_df.shape}, test_df.shape: {test_df.shape}, label_df.shape: {label_df.shape}")
# 处理训练数据和测试数据
train = train_df.copy(deep=True).iloc[:, 1:]
test = test_df.copy(deep=True).iloc[:, 1:]
label = label_df.copy(deep=True)
_, min_a, max_a = normalize3(np.concatenate((train, test), axis=0))
train, _, _ = normalize3(train, min_a, max_a)
test, _, _ = normalize3(test, min_a, max_a)

# 确保数据是数值类型
# train = train.apply(pd.to_numeric, errors='coerce')
# test = test.apply(pd.to_numeric, errors='coerce')
# label = label.apply(pd.to_numeric, errors='coerce')
# 将标签扩展为与 test 相同的维度
label = pd.concat([label] * len(test.columns), axis=1)
# train = convertNumpy(train)
# test, labels = convertNumpy_4(test), convertNumpy_4(label)

labels = label
print(f"train.shape: {train.shape}, test.shape: {test.shape}, labels.shape: {labels.shape}")
output_folder = 'processed/hydrogen'
os.makedirs(output_folder, exist_ok=True)
# 保存数据为.npy文件
for file in ['train', 'test', 'labels']:
    np.save(os.path.join(output_folder, f'{file}.npy'), eval(file))

train.shape: (6770, 98), test.shape: (1786, 98), labels.shape: (1786, 98)
